In [1]:
import os
import torch
import torchvision
import torch.nn as nn
from datetime import datetime

from torch.utils.data import DataLoader
from torch.autograd import Variable

from LipReadDataTrain import ReadData
from LipNet import LipNet, LipSeqLoss

In [3]:
train_image_file = os.path.join(os.path.abspath('.'), "data/lip_train")
train_label_file = os.path.join(os.path.abspath('.'), "data/lip_train.txt")
training_dataset = ReadData(train_image_file, train_label_file, seq_max_lens=24)
training_data_loader = DataLoader(training_dataset, batch_size=20, shuffle=True, num_workers=12, drop_last=True)

# GPU
device = torch.device('cuda:0')
# # CPU 
# device = torch.device('cpu')

model = LipNet().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
loss_fc = LipSeqLoss().to(device)

In [ ]:
for epoch in range(1,1000):
    print(epoch)
    model.train()
    for i_batch, sample_batched in enumerate(training_data_loader):
        
        input_data = Variable(sample_batched['volume']).to(device) 
        labels = Variable(sample_batched['label']).to(device)
        length = Variable(sample_batched['length']).to(device)
        
        optimizer.zero_grad()
        result = model(input_data)  
        loss = loss_fc(result, length, labels)   
        loss.backward()
        optimizer.step()
        
    if epoch % 5 == 0:
        current_time = datetime.now()
        print("current time:", current_time)
        print("number of epoch:", epoch)
        print("current loss:", loss)
                   
        # save model
        torch.save(model.state_dict(), "./weight/demo_net_epoch_{}.pt".format(epoch))

1
